### Introduccion
Implementar un modelo de ML para clasificar si un dominio es legítimo o es Phishing. El phising se basa en la ingeniería social (manipulación de emociones, aprovechamiento de atajos mentales y sesgos cognitivos) para engañar a las víctimas y lograr que estas den información (normalmente credenciales).

## Parte I --- Ingeniería de características

### Explotación de datos

1. Cargue el dataset en un dataframe de pandas, muestre un ejemplo de cinco observaciones.

In [105]:
import pandas as pd

data = pd.read_csv('data/dataset_phishing.csv')
data.head(5)

,url,status
0,http://www.crestonwood.com/router.php,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,phishing
2,https://support-appleld.com.secureupdate.duila...,phishing
3,http://rgipt.ac.in,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,legitimate


2. Muestre la cantidad de observaciones etiquetadas en la columna status como “legit” y como “pishing”. ¿Está balanceado el dataset?

In [106]:
print(data['status'].value_counts())

status
legitimate    5715
phishing      5715
Name: count, dtype: int64


### Derivación de características

1. ¿Qué ventajas tiene el análisis de una URL contra el análisis de otros datos, cómo el tiempo de vida del dominio, o las características de la página Web?

- El análisis de una URL tiene varias ventajas importantes frente a otros tipos de análisis. En primer lugar, es mucho más rápido y ligero, ya que no requiere cargar ni inspeccionar el contenido completo de una página web. Esto permite detectar phishing en tiempo real, incluso antes de que el usuario acceda al sitio. Además, el análisis de URL no depende de servicios externos ni del historial del dominio, lo que lo hace más independiente y resistente ante ataques nuevos. También es más seguro, porque evita interactuar directamente con páginas que pueden ser maliciosas.


2. ¿Qué características de una URL son más prometedoras para la detección de phishing?

- Las características más prometedoras de una URL para detectar phishing son aquellas relacionadas con su estructura y composición. Entre las más útiles se encuentran la longitud de la URL, la cantidad de subdominios, el uso de direcciones IP en lugar de nombres de dominio, la presencia de caracteres especiales, y patrones sospechosos como guiones o múltiples puntos. También resultan muy relevantes las características léxicas y estadísticas, como la frecuencia de símbolos no alfanuméricos o la entropía de estos caracteres. Estas propiedades suelen diferenciar claramente una URL legítima de una maliciosa y permiten a los modelos de machine learning identificar phishing con alta precisión.

### Preprocesamiento
Funciones basadas en los artículos, para derivar características que un modelo pueda utilizar incluidas la entropía de Shanon y relativa

In [107]:
import re
import math
from urllib.parse import urlparse

# 1. Longitud total de la URL
def url_length(url):
    return len(url)

# 2. Número de dígitos en la URL
def digits_len(url):
    return sum(c.isdigit() for c in url)

# 3. Número de subdominios
def num_subdomains(url):
    domain = urlparse(url).netloc
    return domain.count('.') - 1

# 4. Longitud del dominio
def domain_length(url):
    return len(urlparse(url).netloc)

# 5. Longitud del path
def path_length(url):
    return len(urlparse(url).path)

# 6. Presencia de dirección IP
def has_ip(url):
    ip_pattern = r'\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b'
    return 1 if re.search(ip_pattern, url) else 0

# 7. Uso de HTTPS
def uses_https(url):
    return 1 if url.startswith('https') else 0

# 8. Conteo de guiones
def count_hyphens(url):
    return url.count('-')

# 9. Conteo de arrobas (@)
def count_at(url):
    return url.count('@')

# 10. Conteo de puntos
def count_dots(url):
    return url.count('.')

# 11. Palabras sospechosas
def suspicious_words(url):
    words = ['login', 'secure', 'update', 'verify', 'account', 'bank']
    return sum(url.lower().count(word) for word in words)

# 12. Proporción de dígitos
def digit_ratio(url):
    return digits_len(url) / len(url)

# 13. Conteo de caracteres especiales
def special_char_count(url):
    return sum(not c.isalnum() for c in url)

# 14. Entropía de Shannon
def shannon_entropy(url):
    freq = {}
    for c in url:
        freq[c] = freq.get(c, 0) + 1
    entropy = 0
    length = len(url)
    for char in freq:
        p = freq[char] / length
        entropy -= p * math.log2(p)
    return entropy

# 15. Entropía relativa
def relative_entropy(url):
    unique_chars = len(set(url))
    max_entropy = math.log2(unique_chars) if unique_chars > 0 else 1
    return shannon_entropy(url) / max_entropy if max_entropy > 0 else 0

### Aplicando las características

In [108]:
data['url_length'] = data['url'].apply(url_length)

data['digits_len'] = data['url'].apply(digits_len)

data['num_subdomains'] = data['url'].apply(num_subdomains)

data['domain_length'] = data['url'].apply(domain_length)

data['path_length'] = data['url'].apply(path_length)

data['has_ip'] = data['url'].apply(has_ip)

data['uses_https'] = data['url'].apply(uses_https)

data['count_hyphens'] = data['url'].apply(count_hyphens)

data['count_at'] = data['url'].apply(count_at)

data['count_dots'] = data['url'].apply(count_dots)

data['suspicious_words'] = data['url'].apply(suspicious_words)

data['digit_ratio'] = data['url'].apply(digit_ratio)

data['special_char_count'] = data['url'].apply(special_char_count)

data['shannon_entropy'] = data['url'].apply(shannon_entropy)

data['relative_entropy'] = data['url'].apply(relative_entropy)

# Verificar las nuevas columnas
data.head(5)

,url,status,url_length,digits_len,num_subdomains,domain_length,path_length,has_ip,uses_https,count_hyphens,count_at,count_dots,suspicious_words,digit_ratio,special_char_count,shannon_entropy,relative_entropy
0,http://www.crestonwood.com/router.php,legitimate,37,0,1,19,11,0,0,0,0,3,0,0.000000,7,3.787043,0.946761
1,http://shadetreetechnology.com/V4/validation/a...,phishing,77,17,0,23,47,0,0,0,0,1,0,0.220779,7,4.419864,0.909815
2,https://support-appleld.com.secureupdate.duila...,phishing,126,19,3,50,20,0,1,1,0,4,3,0.150794,19,4.753412,0.942316
3,http://rgipt.ac.in,legitimate,18,0,1,11,0,0,0,0,0,2,0,0.000000,5,3.461320,0.965511
4,http://www.iracing.com/tracks/gateway-motorspo...,legitimate,55,0,1,15,33,0,0,2,0,2,0,0.000000,10,4.097662,0.948110


In [109]:
data.info()

<class 'pandas.DataFrame'>
RangeIndex: 11430 entries, 0 to 11429
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   url                 11430 non-null  str    
 1   status              11430 non-null  str    
 2   url_length          11430 non-null  int64  
 3   digits_len          11430 non-null  int64  
 4   num_subdomains      11430 non-null  int64  
 5   domain_length       11430 non-null  int64  
 6   path_length         11430 non-null  int64  
 7   has_ip              11430 non-null  int64  
 8   uses_https          11430 non-null  int64  
 9   count_hyphens       11430 non-null  int64  
 10  count_at            11430 non-null  int64  
 11  count_dots          11430 non-null  int64  
 12  suspicious_words    11430 non-null  int64  
 13  digit_ratio         11430 non-null  float64
 14  special_char_count  11430 non-null  int64  
 15  shannon_entropy     11430 non-null  float64
 16  relative_entrop


### Convertir status en una variable binaria 
- 1 = phising
- 0 = legitima

In [110]:
data['status'] = data['status'].map({'phishing': 1, 'legitimate': 0})
print(data['status'].value_counts())

status
0    5715
1    5715
Name: count, dtype: int64


### Borrando la columna de dominio y guardando el nuevo data set.

In [111]:
new_data = data.drop(columns=['url'])

# Verificar estructura
new_data.head()

,status,url_length,digits_len,num_subdomains,domain_length,path_length,has_ip,uses_https,count_hyphens,count_at,count_dots,suspicious_words,digit_ratio,special_char_count,shannon_entropy,relative_entropy
0,0,37,0,1,19,11,0,0,0,0,3,0,0.000000,7,3.787043,0.946761
1,1,77,17,0,23,47,0,0,0,0,1,0,0.220779,7,4.419864,0.909815
2,1,126,19,3,50,20,0,1,1,0,4,3,0.150794,19,4.753412,0.942316
3,0,18,0,1,11,0,0,0,0,0,2,0,0.000000,5,3.461320,0.965511
4,0,55,0,1,15,33,0,0,2,0,2,0,0.000000,10,4.097662,0.948110


In [112]:
new_data.to_csv('data/new_dataset_phishing.csv', index=False)

### Selección de características

In [113]:
print("Dimensiones del dataset:")
print(new_data.shape)

Dimensiones del dataset:
(11430, 16)


In [114]:
# Verificar observaciones duplicadas

print("\nNúmero de filas duplicadas:")
print(new_data.duplicated().sum())




Número de filas duplicadas:
710


In [115]:
new_data = new_data.drop_duplicates()
print("\nDuplicados eliminados. Nuevo tamaño:")
print(new_data.shape)


Duplicados eliminados. Nuevo tamaño:
(10720, 16)


In [116]:
print("\n--- Columnas con un solo valor único (constantes) ---")

constant_columns = [col for col in new_data.columns if new_data[col].nunique() == 1]

print(constant_columns)


--- Columnas con un solo valor único (constantes) ---
[]


In [117]:
print("\n--- Varianza de cada columna ---")

variances = new_data.var()

print(variances.sort_values())

low_variance = variances[variances < 0.01].index.tolist()

print("\nColumnas con varianza muy baja:")
print(low_variance)

new_data = new_data.drop(columns=low_variance)
print("\nColumnas de baja varianza eliminadas.")


--- Varianza de cada columna ---
relative_entropy         0.000283
digit_ratio              0.008261
has_ip                   0.009242
count_at                 0.025747
shannon_entropy          0.167577
uses_https               0.235429
status                   0.249539
suspicious_words         0.431894
num_subdomains           0.782136
count_dots               1.973426
count_hyphens            4.576577
special_char_count      37.986553
domain_length          120.902787
digits_len             281.980021
path_length            787.917047
url_length            3171.660155
dtype: float64

Columnas con varianza muy baja:
['has_ip', 'digit_ratio', 'relative_entropy']

Columnas de baja varianza eliminadas.


In [118]:
print("\n--- Correlación con status ---")

correlation = new_data.corr()['status'].sort_values()

print(correlation)


--- Correlación con status ---
count_hyphens        -0.127220
uses_https           -0.094682
path_length           0.075443
num_subdomains        0.102832
count_at              0.141306
special_char_count    0.177000
count_dots            0.199070
domain_length         0.226412
url_length            0.228687
digits_len            0.233538
suspicious_words      0.295657
shannon_entropy       0.309567
status                1.000000
Name: status, dtype: float64


### Seleccionado características poco correlacionadas

In [119]:
irrelevant = correlation[abs(correlation) < 0.05].index.tolist()

print("\nColumnas con correlación muy baja con status:")
print(irrelevant)


Columnas con correlación muy baja con status:
[]


In [120]:
print("\n--- Búsqueda de características altamente correlacionadas ---")

corr_matrix = new_data.corr()

to_drop = set()

for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > 0.95:
            colname = corr_matrix.columns[i]
            to_drop.add(colname)

print("\nColumnas altamente correlacionadas entre sí:")
print(to_drop)


--- Búsqueda de características altamente correlacionadas ---

Columnas altamente correlacionadas entre sí:
set()


In [121]:
print("\n--- DATASET FINAL ---")
print("Dimensiones:", new_data.shape)
print("Columnas finales:", new_data.columns)
print("\nVista previa:")
print(new_data.head())
new_data.to_csv('data/new_clean_dataset_phishing.csv', index=False)


--- DATASET FINAL ---
Dimensiones: (10720, 13)
Columnas finales: Index(['status', 'url_length', 'digits_len', 'num_subdomains', 'domain_length',
       'path_length', 'uses_https', 'count_hyphens', 'count_at', 'count_dots',
       'suspicious_words', 'special_char_count', 'shannon_entropy'],
      dtype='str')

Vista previa:
   status  url_length  digits_len  num_subdomains  domain_length  path_length  \
0       0          37           0               1             19           11   
1       1          77          17               0             23           47   
2       1         126          19               3             50           20   
3       0          18           0               1             11            0   
4       0          55           0               1             15           33   

   uses_https  count_hyphens  count_at  count_dots  suspicious_words  \
0           0              0         0           3                 0   
1           0              0         0   

### Columnas seleccionadas

Después del proceso de exploración y limpieza, el dataset quedó con estas 13 características:

['status',
 'url_length',
 'digits_len',
 'num_subdomains',
 'domain_length',
 'path_length',
 'uses_https',
 'count_hyphens',
 'count_at',
 'count_dots',
 'suspicious_words',
 'special_char_count',
 'shannon_entropy']

Se tomaron en cuenta ya que:
- Ninguna de estas columnas tiene un solo valor único. Todas varían entre las URLs, por lo tanto aportan información útil para diferenciar entre phishing y legítimas.
- Tienen varianza suficientemente alta
- Representan patrones relevantes para phishing


## Pate 2 - Implementación

In [123]:
cleaned_data = pd.read_csv('data/new_clean_dataset_phishing.csv')
cleaned_data.shape

(10720, 13)